In [ ]:
# Scraping IMDB Reviews in Python using Selenium // source: https://www.analyticsvidhya.com/blog/2022/07/scraping-imdb-reviews-in-python-using-selenium/

In [1]:
# import libraries
import numpy as np
import pandas as pd
from scrapy.selector import Selector
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
# selenium test
driver = webdriver.Chrome('chromedriver.exe')
url = 'https://www.imdb.com/title/tt0296042/reviews?ref_=tt_urv'
time.sleep(1)
driver.get(url)
time.sleep(1)
print(driver.title)
time.sleep(1)
body = driver.find_element(By.CSS_SELECTOR, 'body')
body.send_keys(Keys.PAGE_DOWN)
time.sleep(1)
body.send_keys(Keys.PAGE_DOWN)
time.sleep(1)
body.send_keys(Keys.PAGE_DOWN)

Ichi the Killer (2001) - Ichi the Killer (2001) - User Reviews - IMDb


In [3]:
# extract review count
sel = Selector(text = driver.page_source)
review_counts = sel.css('.lister .header span::text').extract_first().replace(',','').split(' ')[0]
more_review_pages = int(int(review_counts)/25)

In [33]:
# load all reviews
# I had to run this cell more than one time to get all the re
for i in tqdm(range(more_review_pages)):
    try:
        css_selector = 'load-more-trigger'
        driver.find_element(By.ID, css_selector).click()
    except:
        pass

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 38.25it/s]


In [25]:
# extract review from html
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')
first_review = reviews[0]
sel2 = Selector(text = first_review.get_attribute('innerHTML'))
rating = sel2.css('.rating-other-user-rating span::text').extract_first().strip()

In [26]:
review = sel2.css('.text.show-more__control::text').extract_first().strip()
review_date = sel2.css('.review-date::text').extract_first().strip()
author = sel2.css('.display-name-link a::text').extract_first().strip()
review_title = sel2.css('a.title::text').extract_first().strip()
review_url = sel2.css('a.title::attr(href)').extract_first().strip()
# helpfulness = sel2.css('.actions.text-muted::text').extract_first().strip()
print('nRating:',rating)
print('nreview_title:',review_title)
print('nAuthor:',author)
print('nreview_date:',review_date)
print('nreview:',review)
# print('nhelpfulness:',helpfulness)

nRating: 7
nreview_title: A mixed bag for me
nAuthor: BrandtSponseller
nreview_date: 23 April 2005
nreview: Ichi the Killer is the story of a Yakuza gang run by Anjo, whom his underlings find missing (possibly killed) with 100 million yen gone as well. The functional head of Anjo's gang while he's absent is an off-the-wall sadomasochist named Kakihara (Tadanobu Asano). There is a former affiliate gang, now somewhat rivals, run by Fujiwara (Toru Tezuka), and there are a number of gang outcasts who hang out at a bar/brothel, with Jijii (Shinya Tsukamoto) as their head. Jijii is manipulating/grooming a bizarre killer named Ichi (Nao Omori), who is gradually taking out Anjo and Fujiwara gang members for him.


In [34]:
# putting it all together
rating_list = []
review_date_list = []
review_title_list = []
author_list = []
review_list = []
review_url_list = []
error_url_list = []
error_msg_list = []
reviews = driver.find_elements(By.CSS_SELECTOR, 'div.review-container')

for d in tqdm(reviews):
    try:
        sel2 = Selector(text = d.get_attribute('innerHTML'))
        try:
            rating = sel2.css('.rating-other-user-rating span::text').extract_first()
        except:
            rating = np.NaN
        try:
            review = sel2.css('.text.show-more__control::text').extract_first()
        except:
            review = np.NaN
        try:
            review_date = sel2.css('.review-date::text').extract_first()
        except:
            review_date = np.NaN    
        try:
            author = sel2.css('.display-name-link a::text').extract_first()
        except:
            author = np.NaN    
        try:
            review_title = sel2.css('a.title::text').extract_first()
        except:
            review_title = np.NaN
        try:
            review_url = sel2.css('a.title::attr(href)').extract_first()
        except:
            review_url = np.NaN
        rating_list.append(rating)
        review_date_list.append(review_date)
        review_title_list.append(review_title)
        author_list.append(author)
        review_list.append(review)
        review_url_list.append(review_url)
    except Exception as e:
        error_url_list.append(url)
        error_msg_list.append(e)
review_df = pd.DataFrame({
    'Review_Date':review_date_list,
    'Author':author_list,
    'Rating':rating_list,
    'Review_Title':review_title_list,
    'Review':review_list,
    'Review_Url':review_url
    })

100%|████████████████████████████████████████████████████████████████████████████████| 324/324 [00:04<00:00, 68.63it/s]


In [35]:
review_df.to_csv()


',Review_Date,Author,Rating,Review_Title,Review,Review_Url\r\n0,23 April 2005,BrandtSponseller,7," A mixed bag for me\n","Ichi the Killer is the story of a Yakuza gang run by Anjo, whom his underlings find missing (possibly killed) with 100 million yen gone as well. The functional head of Anjo\'s gang while he\'s absent is an off-the-wall sadomasochist named Kakihara (Tadanobu Asano). There is a former affiliate gang, now somewhat rivals, run by Fujiwara (Toru Tezuka), and there are a number of gang outcasts who hang out at a bar/brothel, with Jijii (Shinya Tsukamoto) as their head. Jijii is manipulating/grooming a bizarre killer named Ichi (Nao Omori), who is gradually taking out Anjo and Fujiwara gang members for him.",/review/rw1328050/?ref_=tt_urv\r\n1,15 September 2003,HumanoidOfFlesh,9," Excellent mix of sadistic violence and humour.\n","Takashi Miike\'s ""Ichi the Killer"" is a masterpiece of insane cinema.This film is surely challenging-filled with enough sadistic violence and 

In [36]:
# save to csv file
compression_opts = dict(method='zip',
                        archive_name='out.csv')  
review_df.to_csv('out.zip', index=False,
          compression=compression_opts)  